In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from superresolution_scripts.superresolution import Superresolution
from superresolution_scripts.optimizer import Optimizer
from utils import load_image, compute_IoU, Mean_IOU, single_class_IOU
from superresolution_scripts.superres_utils import list_precomputed_data_paths, load_SR_data, normalize_coefficients, threshold_image, get_img_paths, filter_images_by_class


In [2]:
IMG_SIZE = (512, 512)
FEATURE_SIZE = (128, 128)
NUM_AUG = 100
CLASS_ID = 8
NUM_SAMPLES = 5

MODE = "argmax"
MODEL_BACKBONE = "xception"
USE_VALIDATION = False
SAVE_SLICE_OUTPUT = False

DATA_DIR = os.path.join(os.getcwd(), "data")
PASCAL_ROOT = os.path.join(DATA_DIR, "dataset_root", "VOCdevkit", "VOC2012")
IMGS_PATH = os.path.join(PASCAL_ROOT, "JPEGImages")

SUPERRES_ROOT = os.path.join(DATA_DIR, "superres_root")
AUGMENTED_COPIES_ROOT = os.path.join(SUPERRES_ROOT, "augmented_copies")
PRECOMPUTED_OUTPUT_DIR = os.path.join(
    AUGMENTED_COPIES_ROOT, f"{MODEL_BACKBONE}_{MODE}_{NUM_AUG}{'_validation' if USE_VALIDATION else ''}")
STANDARD_OUTPUT_ROOT = os.path.join(SUPERRES_ROOT, "standard_output")
STANDARD_OUTPUT_DIR = os.path.join(
    STANDARD_OUTPUT_ROOT, f"{MODEL_BACKBONE}{'_validation' if USE_VALIDATION else ''}")
SUPERRES_OUTPUT_DIR = os.path.join(
    SUPERRES_ROOT, f"superres_output{'_validation' if USE_VALIDATION else ''}")

In [3]:
paths = []

for path, subdirs, files in os.walk(STANDARD_OUTPUT_DIR):
    for filename in files:
        paths.append(os.path.join(path, filename))


paths = sorted(paths, key=lambda p: int(os.path.basename(p).split('.')[0]))
paths = paths[:NUM_SAMPLES]

In [4]:
def test_IoU(y_true, y_pred, classes):
    y_true_squeeze = tf.squeeze(y_true)
    y_pred_squeeze = tf.squeeze(y_pred)
    # classes = [0, CLASS_ID]  # Only check in background and given class
    # y_true_squeeze = tf.where(y_true_squeeze != class_id, 0, y_true_squeeze)

    ious = []
    for i in classes:
        true_labels = tf.equal(y_true_squeeze, i)
        pred_labels = tf.equal(y_pred_squeeze, i)
        inter = tf.cast(true_labels & pred_labels, tf.int32)
        union = tf.cast(true_labels | pred_labels, tf.int32)

        iou = tf.reduce_sum(inter) / tf.reduce_sum(union)
        ious.append(iou)
        print(f"test-{i}-{iou}")

    ious = tf.stack(ious)
    legal_labels = ~tf.math.is_nan(ious)
    ious = tf.gather(ious, indices=tf.where(legal_labels))

    return tf.reduce_mean(ious)

In [5]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count

In [6]:
for k, path in enumerate(paths):
    filename = path.split('\\')[-1]

    ground_truth_path = os.path.join(PASCAL_ROOT, "SegmentationClassAug", filename)
    ground_truth = load_image(ground_truth_path, image_size=IMG_SIZE, normalize=False,
                                  is_png=True, resize_method="nearest")

    standard_mask_path = os.path.join(STANDARD_OUTPUT_DIR, filename)
    standard_mask = load_image(standard_mask_path, image_size=IMG_SIZE, normalize=False, is_png=True,
                                   resize_method="nearest")

    ground_truth = tf.reshape(ground_truth, (IMG_SIZE[0] * IMG_SIZE[1], 1))
    standard_mask = tf.reshape(standard_mask, (IMG_SIZE[0] * IMG_SIZE[1], 1))
    
    # m_iou = Mean_IOU(ground_truth, standard_mask)
    print(filename)
    s_iou = single_class_IOU(ground_truth, standard_mask, class_id=CLASS_ID)
    t_iou = test_IoU(ground_truth, standard_mask, classes=[0, CLASS_ID])
    # print(f"{s_iou}-{t_iou}")


2007_000528.png
void-5992
standard-0-0.9639189304266371
standard-8-0.7630180658873539
test-0-0.9628178098551774
test-8-0.7630180658873539
2007_000549.png
void-4785
standard-0-0.9366122013301584
standard-8-0.9760869789676926
test-0-0.9307924762765477
test-8-0.9760869789676926
2007_000876.png
void-9567
standard-0-0.8707235828255767
standard-8-0.8107324777202163
test-0-0.8840933015679538
test-8-0.8107324777202163
2007_001185.png
void-10014
standard-0-0.9777708826526312
standard-8-0.15813143769041055
test-0-0.5735530346305575
test-8-0.15813143769041055
2007_001825.png
void-15271
standard-0-0.8520903479236812
standard-8-0.6043061063640799
test-0-0.4123531662250436
test-8-0.6043061063640799


# Sweep Data